In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the labels to one-hot encoded vectors
y_train = torch.eye(3)[y_train]
y_test = torch.eye(3)[y_test]

Define the Neurosymbolic model

In [ ]:
class NSPModel(torch.nn.Module):
    def __init__(self):
        super(NSPModel, self).__init__()
        self.fc1 = torch.nn.Linear(4, 8)
        self.fc2 = torch.nn.Linear(8, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

 Define the model

In [ ]:
model = NSPModel()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

Train the model

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(torch.Tensor(X_train))
    loss = criterion(outputs, torch.Tensor(y_train))
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 1000, loss.item()))

Test the model

In [ ]:

with torch.no_grad():
    outputs = model(torch.Tensor(X_test))
    _, predicted = torch.max(outputs.data, 1)
    accuracy = (predicted == torch.max(y_test, 1)[1]).sum().item() / len(y_test)
    print('Accuracy of the model on the test set: {:.2f}%'.format(accuracy * 100))